In [ ]:
!pip install llama-cpp-python
!pip install -U bitsandbytes
!pip install human-eval

In [ ]:
from llama_cpp import Llama
from human_eval.data import write_jsonl, read_problems
from human_eval.evaluation import evaluate_functional_correctness

llm = Llama.from_pretrained(
    repo_id="samzito12/lora_model5", # Specify the Hugging Face repo ID
    filename="model_gguf.gguf",              # Specify the filename pattern for your model file within the repo
    n_gpu_layers=100,                # 4GB GPU: commence à 40 (monte à 60 si stable)
    n_ctx=1024,                     # réduit la mémoire vs 4096
)

def generate_one_completion(prompt: str, max_new_tokens: int = 512) -> str:
    out = llm(
        prompt,
        max_tokens=max_new_tokens,
        temperature=0,
        stop=["\n\n"],
    )
    return out["choices"][0]["text"]


problems = read_problems()
num_samples_per_task = 1  # pass@1
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problems[task_id]["prompt"]))
    for task_id in problems
    for _ in range(num_samples_per_task)
]
write_jsonl("samples.jsonl2", samples)
print("Wrote samples.jsonl2")

results = evaluate_functional_correctness("samples.jsonl2")
print(results)